# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 2 2023 9:42AM,256476,10,9103847,ISDIN Corporation,Released
1,Feb 2 2023 9:39AM,256468,15,30013622,"Alliance Pharma, Inc.",Released
2,Feb 2 2023 9:39AM,256468,15,30013623,"Alliance Pharma, Inc.",Released
3,Feb 2 2023 9:39AM,256468,15,30013624,"Alliance Pharma, Inc.",Released
4,Feb 2 2023 9:39AM,256468,15,30013625,"Alliance Pharma, Inc.",Released
5,Feb 2 2023 9:39AM,256468,15,30013626,"Alliance Pharma, Inc.",Released
6,Feb 2 2023 9:39AM,256468,15,30013627,"Alliance Pharma, Inc.",Released
7,Feb 2 2023 9:39AM,256468,15,30013628,"Alliance Pharma, Inc.",Released
8,Feb 2 2023 9:39AM,256468,15,30013629,"Alliance Pharma, Inc.",Released
9,Feb 2 2023 9:39AM,256468,15,30013630,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,256465,Released,1
29,256466,Released,2
30,256467,Released,23
31,256468,Released,42
32,256476,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256465,NaN,NaN,1.0
256466,NaN,NaN,2.0
256467,NaN,NaN,23.0
256468,NaN,NaN,42.0
256476,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256391,0.0,1.0,0.0
256392,0.0,0.0,1.0
256401,0.0,1.0,0.0
256412,3.0,1.0,0.0
256413,15.0,6.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256391,0,1,0
256392,0,0,1
256401,0,1,0
256412,3,1,0
256413,15,6,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256391,0,1,0
1,256392,0,0,1
2,256401,0,1,0
3,256412,3,1,0
4,256413,15,6,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256391,,1,
1,256392,,,1
2,256401,,1,
3,256412,3,1,
4,256413,15,6,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 2 2023 9:42AM,256476,10,ISDIN Corporation
1,Feb 2 2023 9:39AM,256468,15,"Alliance Pharma, Inc."
43,Feb 2 2023 9:31AM,256467,10,ISDIN Corporation
66,Feb 2 2023 9:29AM,256466,10,ISDIN Corporation
68,Feb 2 2023 9:28AM,256465,10,"Methapharm, Inc."
69,Feb 2 2023 8:57AM,256456,12,SugarBear
70,Feb 2 2023 8:53AM,256454,19,"GCH Granules USA, Inc."
71,Feb 2 2023 8:46AM,256453,16,Sartorius Bioprocess Solutions
72,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation
93,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 2 2023 9:42AM,256476,10,ISDIN Corporation,,,1
1,Feb 2 2023 9:39AM,256468,15,"Alliance Pharma, Inc.",,,42
2,Feb 2 2023 9:31AM,256467,10,ISDIN Corporation,,,23
3,Feb 2 2023 9:29AM,256466,10,ISDIN Corporation,,,2
4,Feb 2 2023 9:28AM,256465,10,"Methapharm, Inc.",,,1
5,Feb 2 2023 8:57AM,256456,12,SugarBear,,,1
6,Feb 2 2023 8:53AM,256454,19,"GCH Granules USA, Inc.",,,1
7,Feb 2 2023 8:46AM,256453,16,Sartorius Bioprocess Solutions,,,1
8,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,,10,11
9,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 9:42AM,256476,10,ISDIN Corporation,1,,
1,Feb 2 2023 9:39AM,256468,15,"Alliance Pharma, Inc.",42,,
2,Feb 2 2023 9:31AM,256467,10,ISDIN Corporation,23,,
3,Feb 2 2023 9:29AM,256466,10,ISDIN Corporation,2,,
4,Feb 2 2023 9:28AM,256465,10,"Methapharm, Inc.",1,,
5,Feb 2 2023 8:57AM,256456,12,SugarBear,1,,
6,Feb 2 2023 8:53AM,256454,19,"GCH Granules USA, Inc.",1,,
7,Feb 2 2023 8:46AM,256453,16,Sartorius Bioprocess Solutions,1,,
8,Feb 2 2023 8:34AM,256451,10,ISDIN Corporation,11,10,
9,Feb 2 2023 8:33AM,256448,10,ISDIN Corporation,29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 9:42AM,256476,10,ISDIN Corporation,1,,
1,Feb 2 2023 9:39AM,256468,15,"Alliance Pharma, Inc.",42,,
2,Feb 2 2023 9:31AM,256467,10,ISDIN Corporation,23,,
3,Feb 2 2023 9:29AM,256466,10,ISDIN Corporation,2,,
4,Feb 2 2023 9:28AM,256465,10,"Methapharm, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 9:42AM,256476,10,ISDIN Corporation,1.0,NaN,NaN
1,Feb 2 2023 9:39AM,256468,15,"Alliance Pharma, Inc.",42.0,NaN,NaN
2,Feb 2 2023 9:31AM,256467,10,ISDIN Corporation,23.0,NaN,NaN
3,Feb 2 2023 9:29AM,256466,10,ISDIN Corporation,2.0,NaN,NaN
4,Feb 2 2023 9:28AM,256465,10,"Methapharm, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 9:42AM,256476,10,ISDIN Corporation,1.0,0.0,0.0
1,Feb 2 2023 9:39AM,256468,15,"Alliance Pharma, Inc.",42.0,0.0,0.0
2,Feb 2 2023 9:31AM,256467,10,ISDIN Corporation,23.0,0.0,0.0
3,Feb 2 2023 9:29AM,256466,10,ISDIN Corporation,2.0,0.0,0.0
4,Feb 2 2023 9:28AM,256465,10,"Methapharm, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3077398,95.0,12.0,6.0
12,769271,2.0,1.0,0.0
15,256468,42.0,0.0,0.0
16,256453,1.0,0.0,0.0
18,256436,1.0,0.0,0.0
19,1538562,8.0,8.0,18.0
21,512783,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3077398,95.0,12.0,6.0
1,12,769271,2.0,1.0,0.0
2,15,256468,42.0,0.0,0.0
3,16,256453,1.0,0.0,0.0
4,18,256436,1.0,0.0,0.0
5,19,1538562,8.0,8.0,18.0
6,21,512783,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,95.0,12.0,6.0
1,12,2.0,1.0,0.0
2,15,42.0,0.0,0.0
3,16,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,8.0,8.0,18.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,95.0
1,12,Released,2.0
2,15,Released,42.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21
Status,,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,18.0,0.0
Executing,12.0,1.0,0.0,0.0,0.0,8.0,1.0
Released,95.0,2.0,42.0,1.0,1.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21
0,Completed,6.0,0.0,0.0,0.0,0.0,18.0,0.0
1,Executing,12.0,1.0,0.0,0.0,0.0,8.0,1.0
2,Released,95.0,2.0,42.0,1.0,1.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21
0,Completed,6.0,0.0,0.0,0.0,0.0,18.0,0.0
1,Executing,12.0,1.0,0.0,0.0,0.0,8.0,1.0
2,Released,95.0,2.0,42.0,1.0,1.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()